# Facial Expression Recognition Dataset overview

This example is using the dataset from the Kaggle competition (https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/leaderboard). The data consists of 48x48 pixel grayscale images of faces. The task is to categorize each face based on the emotion shown in the facial expression in to one of seven categories (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral).

# Convolutional Neural Network

<img src="http://personal.ie.cuhk.edu.hk/~ccloy/project_target_code/images/fig3.png">

This CNN Architecture has been used to predict the facial expressions.

# Importing libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib.pyplot as plt 
import seaborn as sns
import os
from PIL import Image
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.utils import np_utils


import keras
from keras.layers import Dense, Conv2D
from keras.layers import Flatten
from keras.layers import MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.models import Sequential
from keras import backend as K
from keras import optimizers

Using TensorFlow backend.


In [73]:

data_dict = { }

class_number = 0

data = []

labels = []

# Extracting file names into a dictionary 
for i in range(7): 
    
    data_dict["class_" + str(i)] = os.listdir('Training/'+str(i))
    

# Converting the images into array (input) and label (outputs) 
for emotion_class,img_names in data_dict.items():
    
    for img in img_names:
        
        img_read = plt.imread('Training/'+ str(class_number) +  '/' + img)
        img_resize = cv2.resize(img_read, (48, 48)) # Converting image to (48, 48)
        img_array = img_to_array(img_resize) # Converting to array
        data.append(img_array)
        labels.append(class_number)
        
    class_number = class_number+1

In [38]:
image_data = np.array(data)

labels = np.array(labels)

In [39]:
# Shuffling the data
idx = np.arange(image_data.shape[0])

np.random.shuffle(idx)

image_data = image_data[idx]

labels = labels[idx]

In [41]:
# Splitting data into testing and training sets

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(image_data, labels, test_size = 0.2, random_state = 101)

In [42]:
y_train = np_utils.to_categorical(y_train, num_classes = 7)

y_test = np_utils.to_categorical(y_test, num_classes = 7)

In [66]:
# Building a model

def CNNbuild(height, width, classes, channels):
    model = Sequential()
    
    inputShape = (height, width, channels)
    chanDim = -1
    
    
    if K.image_data_format() == 'channels_first':
        inputShape = (channels, height, width)
    model.add(Conv2D(64, (3,3), activation = 'relu', input_shape = inputShape))
    model.add(MaxPooling2D(2,2))
    model.add(BatchNormalization(axis = chanDim))
    model.add(Dropout(0.4))

    model.add(Conv2D(128, (3,3), activation = 'relu'))
    model.add(MaxPooling2D(2,2))
    model.add(BatchNormalization(axis = chanDim))
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3,3), activation = 'relu'))
    model.add(MaxPooling2D(2,2))
    model.add(BatchNormalization(axis = chanDim))
    model.add(Dropout(0.4))

    model.add(Flatten())
    
    model.add(Dense(1024, activation = 'relu'))
    model.add(BatchNormalization(axis = chanDim))
    model.add(Dropout(0.5))
    model.add(Dense(classes, activation = 'softmax'))
    
    return model

In [67]:
height = 48 # Resized image size (48, 48)
width = 48
classes = 7 # 7 output classes
channels = 1 # Monochrome images so single channel 
model = CNNbuild(height = height, width = width, classes = classes, channels = channels)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 46, 46, 64)        640       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 23, 23, 64)        256       
_________________________________________________________________
dropout_9 (Dropout)          (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 21, 21, 128)       73856     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 10, 10, 128)       512       
__________

In [68]:
#Compile the model

model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

In [69]:
h = model.fit(x_train, y_train, epochs = 10, batch_size = 32)


Epoch 1/10
22966/22966 [==============================] - 519s 23ms/step - loss: 2.1035 - acc: 0.3160
Epoch 2/10
22966/22966 [==============================] - 551s 24ms/step - loss: 1.5926 - acc: 0.4182
Epoch 3/10
22966/22966 [==============================] - 638s 28ms/step - loss: 1.4568 - acc: 0.4540
Epoch 4/10
22966/22966 [==============================] - 680s 30ms/step - loss: 1.3718 - acc: 0.4820
Epoch 5/10
22966/22966 [==============================] - 636s 28ms/step - loss: 1.3188 - acc: 0.5073
Epoch 6/10
22966/22966 [==============================] - 572s 25ms/step - loss: 1.2706 - acc: 0.5233
Epoch 7/10
22966/22966 [==============================] - 585s 25ms/step - loss: 1.2235 - acc: 0.5371
Epoch 8/10
22966/22966 [==============================] - 583s 25ms/step - loss: 1.1911 - acc: 0.5504
Epoch 9/10
22966/22966 [==============================] - 639s 28ms/step - loss: 1.1522 - acc: 0.5671
Epoch 10/10
22966/22966 [==============================] - 697s 30ms/step - loss: 

In [70]:
predictions = model.evaluate(x_test, y_test)

5742/5742 [==============================] - 25s 4ms/step


In [71]:
print(f'LOSS : {predictions[0]}')
print(f'ACCURACY : {predictions[1]}')

LOSS : 1.1467532117123473
ACCURACY : 0.5689655173036621


# CNN Output


EPOCS : 10

LOSS : 1.1467532117123473

TESTING ACCURACY : 0.5689655173036621

TRAINING ACCURACY : 0.5788
